In [7]:
from pdp_utils import *
prob = load_problem('./Call_7_Vehicle_3.txt')
print(prob.keys())


dict_keys(['n_nodes', 'n_vehicles', 'n_calls', 'Cargo', 'TravelTime', 'FirstTravelTime', 'VesselCapacity', 'LoadingTime', 'UnloadingTime', 'VesselCargo', 'TravelCost', 'FirstTravelCost', 'PortCost'])
[[0. 1. 1. 1. 1. 0. 1.]
 [0. 1. 1. 0. 1. 0. 1.]
 [1. 1. 1. 0. 1. 1. 1.]]


Pull random call from prob["Cargo"]. 

Assign call to random vehicle.

Vehicle turns up late to destination: solution is valid but infeasible.

In [3]:
import random
import functools

def forward():

    vehicle_carry = np.zeros(prob["n_vehicles"] + 1)
    vehicle_time = np.zeros(prob["n_vehicles"] + 1)
    vehicle_cost = np.zeros(prob["n_vehicles"] + 1)
    vehicle_pos = np.zeros(prob["n_vehicles"] + 1)

    vehicle_actions = [[-1] for i in range(int(prob["n_vehicles"]) + 1)]
    active_vehicles = [True for i in range(int(prob["n_vehicles"]))]

    np.random.seed(0)
    n_vehicles = prob["n_vehicles"]
    cargos = prob["Cargo"].copy()
    max_time = np.max(cargos, axis=0)[7]

    while np.size(cargos, 0) > 0:

        # Draw random cargo from unanswered pile
        to_select = np.random.choice(np.size(cargos, 0))
        current_cargo = cargos[to_select]
        cargos = np.delete(cargos, to_select, axis=0)
        true_idx = np.where(np.all(prob["Cargo"] == current_cargo,axis=1))[0][0]

        # Draw random vehicle that has not reached end of schedule.
        cand_v = [i for i, x in enumerate(zip(active_vehicles, prob["VesselCargo"])) if x[0] and x[1][true_idx] == 1.0]
        v_idx = np.random.choice(cand_v)    

        #Add to vehicle time from previous location to current location, adjust for waiting.
        vehicle_time[v_idx] = max(prob["FirstTravelTime"][v_idx][int(current_cargo[0])], current_cargo[4]) if vehicle_time[v_idx] == 0 else vehicle_time[v_idx] + max(prob["TravelTime"][v_idx][int(vehicle_pos[v_idx])][int(current_cargo[1])], current_cargo[4])

        # If vehicle is past max time, give cargo to dummy and deactivate vehicle. 
        if vehicle_time[v_idx] >= current_cargo[7]:
            active_vehicles[v_idx] = False
            v_idx = int(prob["n_vehicles"])
            vehicle_actions[v_idx].append(true_idx)
            vehicle_cost[v_idx] += current_cargo[3]
            continue
        # Add cost for driving from old cargo end to new cargo start
        # Currently adds wrong index as indexes are shifted when items are removed

        vehicle_actions[v_idx].append(true_idx)
        vehicle_cost[v_idx] = prob["FirstTravelCost"][v_idx][int(current_cargo[0])] if vehicle_time[v_idx] == 0 else vehicle_cost[v_idx] + prob["TravelCost"][v_idx][int(vehicle_pos[v_idx])][int(current_cargo[1])]
        # Update vehicle position, payload.
        vehicle_pos[v_idx] = current_cargo[0]
        vehicle_carry[v_idx] += current_cargo[2]

        # Add time for driving from cargo start to cargo end
        vehicle_time[v_idx] = max(vehicle_time[v_idx] + prob["TravelTime"][v_idx][int(vehicle_pos[v_idx])][int(current_cargo[1])], current_cargo[6])
        # Add cost for driving from cargo start to cargo end
        vehicle_cost[v_idx] += prob["TravelCost"][v_idx][int(vehicle_pos[v_idx])][int(current_cargo[1])]
        # Remove carry weight as we are dropping off what we picked up
        vehicle_carry[v_idx] -= current_cargo[2]

    solution = [[y + 1 for y in x if y >= 0] for x in vehicle_actions]
    solution = functools.reduce(lambda acc, cur: acc + cur + [0], solution, [])
    feasiblity, c = feasibility_check(solution, prob)
    return solution, feasiblity, c, vehicle_cost
sol, feas, c, v_cos = forward()
print(c)

        


Time window exceeded at call 1


In [41]:
import time

start = time.time()
init_solution, init_feasibility, init_c, init_vehicle_cost = forward()
best_solution = init_solution
best_cost = np.sum(init_vehicle_cost)
total_cost = best_cost
feasibles = 0
for i in range(10):
    for epoch in range(10000):
        solution, feasibility, c, vehicle_cost = forward()
        cost = np.sum(vehicle_cost)
        total_cost += cost
        if feasibility and cost < best_cost:
            feasibles += 1
            best_cost = cost
            best_solution = solution
avg_cost = total_cost / 100000
print("best cost", best_cost)
print("avg cost ", avg_cost) 
print("% feasible:", (feasibles / 100000))

end = time.time()
print("Time elapsed", (end - start))      

best cost 922192.0
avg cost  922201.22192
% feasible: 0.0
Time elapsed 81.60944485664368


In [42]:
print(feasibles)

0
